In [1]:
from pyomo.environ import *
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from dash import Dash, dcc, html
import webbrowser
from threading import Timer
import dash_bootstrap_components as dbc
import plotly.io as pio
from dash.dependencies import Input, Output
import json
from scripts.result_export import generate_html
from scripts.price_process import price_process


# Read Data

In [2]:
excel_mode = 2

In [3]:
if excel_mode == 1:
    # Read price data from Excel
    price_data = pd.read_excel('data/price_data.xlsx', sheet_name= None, index_col=0)

    data = price_data

    # Iteratively merge all dataframes on 'Timestamp'
    price_df = pd.DataFrame(index =  data[list(data.keys())[0]].index)

    for key in data:
        price_df = pd.concat([price_df, data[key]], axis = 1)

    model_time_period = 24    # periods
    dt = 1



In [4]:
if excel_mode != 1:
    dir_path = 'data/prices/'
    prim_category = 'PRIRESC'
    cons_category = 'CONRESC'

    price_df = price_process(dir_path, prim_category, cons_category)

    model_time_period = 48    # periods
    dt = 0.5


Found 12 CSV files in data/prices/energy/
Found 12 CSV files in data/prices/regulation/
Found 12 CSV files in data/prices/reserve/


In [5]:
# Parameters
schedule_data = pd.read_excel('data/schedule_data_s3.xlsx', sheet_name= None, index_col=0)

λ_min = schedule_data['soc_limit']['min']
λ_max = schedule_data['soc_limit']['max']
theta = schedule_data['availability']

z = {}
s = {}
z['arb'], z['reg'], z['pres'], z['cres'], z['opp'] = schedule_data['basis'].value.values
s['reg'] = schedule_data['basis'].loc['regulation', 'sym']


In [6]:
filename = "test"

# z['opp'] = 0
# s['reg'] = 1

Seff = 0.9995
Ceff = 0.9
Deff = 0.9
cap_power = 1  # Example value
cap_energy = 4  # Example value

λ_0 = 0.5
initial_soc = λ_0 * cap_energy

kappa = 0.95     # regulation performance score
beta = 0.15       # activated portion for reserve


cycle_life = 3650
calendar_life = 10
total_time_period = len(price_df)
num_slices = int(total_time_period // model_time_period)     # number of time slices to model

η = cycle_life/calendar_life/365                 # number of cycle per time slice

# Model

In [7]:
def optimize_revenue(initial_soc, price_vector, current_period, first_period, last_period):

    # Define the model
    model = ConcreteModel()

    # Price Data for the day
    total_time_period = len(price_vector)

    T = range(1, total_time_period + 1)
    SOC_T = range(0, total_time_period + 1)

    p_arb = {t: price_vector['arb_energy_price'].values[t-1] for t in T}
    p_pres = {t: price_vector['pres_capacity_price'].values[t-1] for t in T}
    p_cres = {t: price_vector['cres_capacity_price'].values[t-1] for t in T}

    # p_reg = {t: price_vector['reg_capacity_price'].values[t-1] for t in T}
    p_reg_down = {t: price_vector['reg_down_price'].values[t-1] for t in T}
    p_reg_up = {t: price_vector['reg_up_price'].values[t-1] for t in T}

    start_soc = λ_0 * cap_energy

    # Variables
    model.SOC = Var(SOC_T, within=NonNegativeReals, bounds = (0, cap_energy))
 
    model.C_arb = Var(T, within=NonNegativeReals, bounds = (0, 999))
    model.D_arb = Var(T, within=NonNegativeReals, bounds = (0, 999))
    model.C_reg = Var(T, within=NonNegativeReals, bounds = (0, 999))
    model.D_reg = Var(T, within=NonNegativeReals, bounds = (0, 999))
    model.D_pres = Var(T, within=NonNegativeReals, bounds = (0, 999))
    model.D_cres = Var(T, within=NonNegativeReals, bounds = (0, 999))

    model.y_ch = Var(T, within=Binary)
    model.y_minus = Var(T, within=Binary)

    model.R_arb = Var(T, within=Reals)
    model.R_reg = Var(T, within=Reals)
    model.R_pres = Var(T, within=Reals)
    model.R_cres = Var(T, within=Reals)

    # Objective function
    model.obj = Objective(
        expr= sum(model.R_arb[t] + model.R_reg[t] + model.R_pres[t] + model.R_cres[t] for t in T),        
        sense=maximize
    )

    def revenue_arbitrage_rule(model, t):
        return model.R_arb[t] == p_arb[t]  * (model.D_arb[t] - model.C_arb[t]) * dt
    model.revenue_arbitrage = Constraint(T, rule=revenue_arbitrage_rule)

    def revenue_primary_reserve_rule(model, t):
        return model.R_pres[t] == p_pres[t]  * model.D_pres[t] * dt
    model.revenue_preserve = Constraint(T, rule=revenue_primary_reserve_rule)

    def revenue_contingency_reserve_rule(model, t):
        return model.R_cres[t] == p_cres[t]  * model.D_cres[t] * dt
    model.revenue_creserve = Constraint(T, rule=revenue_contingency_reserve_rule)

    # Arbitrage Constraints

    def c_limit_constraint(model, t):
        return model.C_arb[t] <= cap_power * model.y_ch[t]
    model.C_limit = Constraint(T, rule=c_limit_constraint)

    def d_limit_constraint(model, t):
        return model.D_arb[t]  <= cap_power * (1 - model.y_ch[t])
    model.D_limit = Constraint(T, rule=d_limit_constraint)

    def soc_constraints(model, t):
        if t == 0:
            return model.SOC[t] == initial_soc
        else:
            # return model.SOC[t] == model.SOC[t-1] * Seff + ((model.C_arb[t] + 0.15 * model.C_reg[t]) * Ceff - (model.D_arb[t] + 0.15 * model.D_reg[t])/Deff) * dt
            return model.SOC[t] == model.SOC[t-1] * Seff + (model.C_arb[t]  * Ceff - model.D_arb[t]/Deff) * dt
    model.SOC_constraints = Constraint(SOC_T, rule=soc_constraints)

    def soc_lower_limit_constraint(model, t):
        return model.SOC[t] >= cap_energy * λ_min[t]
    model.SOC_lower_limit = Constraint(T, rule=soc_lower_limit_constraint)

    def soc_upper_limit_constraint(model, t):
        return model.SOC[t] <= cap_energy * λ_max[t]
    model.SOC_upper_limit = Constraint(T, rule=soc_upper_limit_constraint)

    if current_period == last_period:
        def soc_final_state_constraint(model):
            end_t = SOC_T[-1]
            return model.SOC[end_t] == start_soc 
        model.SOC_final = Constraint(rule=soc_final_state_constraint)


    # Frequency and reserve constraints
    def total_charge_rule(model, t):
        return model.C_reg[t] - (model.D_arb[t] - model.C_arb[t])  <= cap_power 
    model.total_charge = Constraint(T, rule=total_charge_rule)

    def total_discharge_rule(model, t):
        return model.D_reg[t] + (model.D_arb[t] - model.C_arb[t]) +  model.D_pres[t] +  model.D_cres[t] <= cap_power 
    model.total_discharge = Constraint(T, rule=total_discharge_rule)

    def soc_reg_down_rule(model, t):
        return model.SOC[t] + beta * model.C_reg[t] * Ceff * dt <= cap_energy
    model.soc_reg_down = Constraint(T, rule=soc_reg_down_rule)

    def soc_reg_up_rule(model, t):
        return model.SOC[t] >= (beta * model.D_reg[t] + model.D_pres[t] + model.D_cres[t])/Deff * dt
    model.soc_reg_up = Constraint(T, rule=soc_reg_up_rule)

    if s['reg'] == 1:
        def reg_constraint_rule(model, t):
            return model.C_reg[t] == model.D_reg[t]
        model.reg_up_down = Constraint(T, rule=reg_constraint_rule)

        def revenue_regulation_rule(model, t):
            return model.R_reg[t] == kappa * (p_reg_up[t] * model.D_reg[t]) * dt
        model.revenue_regulation = Constraint(T, rule=revenue_regulation_rule)

    else:

        def revenue_regulation_rule(model, t):
            return model.R_reg[t] == kappa * (p_reg_down[t] * model.C_reg[t] + p_reg_up[t] * model.D_reg[t]) * dt
        model.revenue_regulation = Constraint(T, rule=revenue_regulation_rule)


    if z['opp'] == 0:
        def total_charge_no_opp_rule(model, t):
            return model.C_reg[t]  + model.C_arb[t]  <= cap_power * model.y_minus[t]
        model.total_charge_nopp = Constraint(T, rule=total_charge_no_opp_rule)

        def total_discharge_no_opp_rrule(model, t):
            return model.D_reg[t] + model.D_arb[t] +  model.D_pres[t] +  model.D_cres[t] <= cap_power * (1 - model.y_minus[t])
        model.total_discharge_nopp = Constraint(T, rule=total_discharge_no_opp_rrule)

    # Service availability

    def arbitrage_availability_rule(model, t):
        return model.C_arb[t] + model.D_arb[t] <= cap_power * z['arb'] * theta['arb'].loc[t] * 10
    model.arbitrage_availability = Constraint(T, rule=arbitrage_availability_rule)

    def regulation_availability_rule(model, t):
        return model.C_reg[t] + model.D_reg[t] <= cap_power * z['reg'] * theta['reg'].loc[t] * 10
    model.regulation_availability = Constraint(T, rule=regulation_availability_rule)

    def preserve_availability_rule(model, t):
        return model.D_pres[t] <= cap_power * z['pres'] * theta['pres'].loc[t] * 10
    model.preserve_availability = Constraint(T, rule=preserve_availability_rule)

    def creserve_availability_rule(model, t):
        return model.D_cres[t] <= cap_power * z['cres'] * theta['cres'].loc[t] * 10
    model.creserve_availability = Constraint(T, rule=creserve_availability_rule)


    # Cycle limits

    # def energy_cap_C_constraint(model):
    #     return sum(model.C_arb[t] +  model.C_reg[t]*beta for t in T)*dt <= η * cap_energy
    #     # return sum(model.C_arb[t] for t in T)*dt <= η * cap_energy
    # model.energy_cap_C = Constraint(rule=energy_cap_C_constraint)

    def energy_cap_D_constraint(model):
        return sum(model.D_arb[t] + model.D_reg[t]*beta for t in T)*dt <= η * cap_energy
        # return sum(model.D_arb[t] for t in T)*dt <= η * cap_energy
    model.energy_cap_D = Constraint(rule=energy_cap_D_constraint)


    # Solve the model
    solver = SolverFactory('glpk')
    solver.solve(model, tee=True)

    # Extract results for charging, discharging, and SOC
    charging_schedule = [model.C_arb[t].value for t in T]  # Charging as negative
    discharging_schedule = [model.D_arb[t].value for t in T]  # Discharging as positive
    reg_down_schedule = [model.C_reg[t].value for t in T]  # Charging as negative
    reg_up_schedule = [model.D_reg[t].value for t in T]  # Discharging as positive
    pres_schedule = [model.D_pres[t].value for t in T]  # Charging as negative
    cres_schedule = [model.D_cres[t].value for t in T]  # Discharging as positive

    soc_schedule = [model.SOC[t].value for t in SOC_T]  # State of Charge

    rev_arbitrage =  sum(model.R_arb[t] for t in T)() 
    rev_regulation = sum(model.R_reg[t] for t in T)() 
    rev_prim_reserve = sum(model.R_pres[t] for t in T)() 
    rev_cont_reserve = sum(model.R_cres[t] for t in T)() 

    # Return the results and final SOC
    return model.obj(), [charging_schedule, discharging_schedule, soc_schedule, reg_down_schedule, reg_up_schedule, pres_schedule, cres_schedule], [rev_arbitrage, rev_regulation, rev_prim_reserve, rev_cont_reserve]

# Run

In [8]:
# Lists to store results
total_revenue = 0
arbitrage = 0
regulation = 0
preserve = 0
creserve = 0

all_charging_schedules = []
all_discharging_schedules = []
all_reg_down_schedules = []
all_reg_up_schedules = []
all_pres_schedules = []
all_cres_schedules = []
all_soc_schedules = []

first_p = 0
last_p = num_slices #- 1

# Run the optimization for each time period
for p in range(num_slices):
    print("watch", p)
    periodic_price = price_df[p*model_time_period:(p+1)*model_time_period]
    revenue, [charging_schedule, discharging_schedule, soc_schedule, reg_down_schedule, reg_up_schedule, pres_schedule, cres_schedule], [rev_arbitrage, rev_regulation, rev_prim_reserve, rev_cont_reserve] = optimize_revenue(initial_soc, periodic_price, p, first_p, last_p)
    final_soc = soc_schedule[-1]
    
    # Store the results
    total_revenue += revenue
    arbitrage += rev_arbitrage
    regulation += rev_regulation
    preserve += rev_prim_reserve
    creserve += rev_cont_reserve

    all_charging_schedules.extend(charging_schedule)
    all_discharging_schedules.extend(discharging_schedule)
    all_reg_down_schedules.extend(reg_down_schedule)
    all_reg_up_schedules.extend(reg_up_schedule)
    all_pres_schedules.extend(pres_schedule)
    all_cres_schedules.extend(cres_schedule)
    all_soc_schedules.extend(soc_schedule[:-1])

    # Update the initial SOC for the next day
    initial_soc = final_soc

print(total_revenue)

watch 0
GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /var/folders/zm/q5z_sctj6kqfn_ln7_6g3wxh0000gn/T/tmpuxpr21aw.glpk.raw
 --wglp /var/folders/zm/q5z_sctj6kqfn_ln7_6g3wxh0000gn/T/tmpviviokm3.glpk.glp
 --cpxlp /var/folders/zm/q5z_sctj6kqfn_ln7_6g3wxh0000gn/T/tmp19b4vg58.pyomo.lp
Reading problem data from '/var/folders/zm/q5z_sctj6kqfn_ln7_6g3wxh0000gn/T/tmp19b4vg58.pyomo.lp'...
/var/folders/zm/q5z_sctj6kqfn_ln7_6g3wxh0000gn/T/tmp19b4vg58.pyomo.lp:5250: warning: lower bound of variable 'x531' redefined
/var/folders/zm/q5z_sctj6kqfn_ln7_6g3wxh0000gn/T/tmp19b4vg58.pyomo.lp:5250: warning: upper bound of variable 'x531' redefined
818 rows, 577 columns, 2017 non-zeros
48 integer variables, all of which are binary
5298 lines were read
Writing problem data to '/var/folders/zm/q5z_sctj6kqfn_ln7_6g3wxh0000gn/T/tmpviviokm3.glpk.glp'...
4714 lines were written
GLPK Integer Optimizer 5.0
818 rows, 577 columns, 2017 non-zeros
48 integer variables, all of which 

In [9]:
revenue_data = pd.Series()
revenue_data.loc['Arbitrage'] = arbitrage/num_slices * 365
revenue_data.loc['Regulation'] = regulation/num_slices * 365
revenue_data.loc['Primary Reserve'] = preserve/num_slices * 365
revenue_data.loc['Contingency Reserve'] = creserve/num_slices * 365

pd.DataFrame(revenue_data, columns = ['revenue'])

,revenue
Arbitrage,435082.649312
Regulation,628737.815352
Primary Reserve,226.728034
Contingency Reserve,28938.090170


In [10]:
df_filtered = revenue_data[revenue_data.values > 0]

fig = go.Figure(data=[go.Pie(labels=df_filtered.index, values=df_filtered.values, hole=0.3)])

# Update layout for the pie chart
fig.update_layout(
    title_text='Breakdown of Annual Revenues',
    annotations=[dict(text=' ', x=0.5, y=0.5, font_size=20, showarrow=False)]
)

# Display the plot in a notebook or interactive environment
fig.show()

In [11]:
# Create a DataFrame for the results
data = {
    'time': price_df.index,
    'charge': all_charging_schedules,
    'discharge': all_discharging_schedules,
    'reg_down': all_reg_down_schedules,
    'reg_up': all_reg_up_schedules,
    'pres': all_pres_schedules,
    'cres': all_cres_schedules,
    'soc': all_soc_schedules
}

result_df = pd.DataFrame(data)

#result_df = result_df.set_index(['t'])

result_df['net_discharge'] = result_df['discharge'] - result_df['charge']
result_df['soc_percent'] = result_df['soc'] /cap_energy

result_df

,time,charge,discharge,reg_down,reg_up,pres,cres,soc,net_discharge,soc_percent
0,2023-01-01 00:00:00,0.0,0.00000,1.00000,0.0000,1.0,0.0,2.000000,0.00000,0.500000
1,2023-01-01 00:30:00,0.0,0.00000,1.00000,0.0000,1.0,0.0,1.999000,0.00000,0.499750
2,2023-01-01 01:00:00,0.0,0.00000,1.00000,0.0000,1.0,0.0,1.998001,0.00000,0.499500
3,2023-01-01 01:30:00,0.0,0.00000,1.00000,0.0000,1.0,0.0,1.997001,0.00000,0.499250
4,2023-01-01 02:00:00,0.0,0.00000,1.00000,0.0000,1.0,0.0,1.996003,0.00000,0.499001
...,...,...,...,...,...,...,...,...,...,...
16987,2023-12-20 21:30:00,0.0,0.00000,1.00000,1.0000,0.0,0.0,0.083458,0.00000,0.020865
16988,2023-12-20 22:00:00,0.0,0.00000,1.00000,1.0000,0.0,0.0,0.083417,0.00000,0.020854
16989,2023-12-20 22:30:00,0.0,0.00000,1.00000,1.0000,0.0,0.0,0.083375,0.00000,0.020844
16990,2023-12-20 23:00:00,0.0,0.00000,1.00000,0.9995,0.0,0.0,0.083333,0.00000,0.020833


In [12]:
result_df[result_df.cres != 0]

,time,charge,discharge,reg_down,reg_up,pres,cres,soc,net_discharge,soc_percent
54,2023-01-02 03:00:00,1.0,0.000000,-4.440892e-16,0.000000,2.0,4.440892e-16,0.899775,-1.000000,0.224944
55,2023-01-02 03:30:00,1.0,0.000000,1.110223e-16,0.000000,2.0,-1.110223e-16,1.349325,-1.000000,0.337331
58,2023-01-02 05:00:00,1.0,0.000000,-2.220446e-16,0.000000,2.0,2.220446e-16,2.696627,-1.000000,0.674157
114,2023-01-03 09:00:00,0.0,0.000000,1.000000e+00,0.000000,0.0,1.000000e+00,2.351814,0.000000,0.587954
116,2023-01-03 10:00:00,0.0,0.000000,1.000000e+00,0.000000,0.0,1.000000e+00,2.349463,0.000000,0.587366
...,...,...,...,...,...,...,...,...,...,...
16853,2023-12-18 02:30:00,1.0,0.000000,0.000000e+00,0.000000,0.0,2.000000e+00,0.884542,-1.000000,0.221136
16928,2023-12-19 16:00:00,0.0,0.000000,1.000000e+00,0.000000,0.0,1.000000e+00,0.656414,0.000000,0.164103
16929,2023-12-19 16:30:00,0.0,0.028961,1.028961e+00,0.000000,0.0,9.710394e-01,0.656086,0.028961,0.164021
16931,2023-12-19 17:30:00,0.0,0.000000,1.000000e+00,0.999115,0.0,8.847843e-04,0.083793,0.000000,0.020948


In [13]:
price_df.index.names = ['time']
price_data_ex = price_df.reset_index().to_json(orient='records', date_format='iso')
result_data_ex = result_df.to_json(orient='records', date_format='iso')

In [14]:
initial_cost = 207000 + 355000 * cap_energy + 153000*cap_power
annual_cost = initial_cost * 0.05
annual_revenue = total_revenue/num_slices * 365
discount_rate = 0.05

generate_html(filename, initial_cost, annual_cost, annual_revenue, price_data_ex, result_data_ex, discount_rate)


HTML file generated:test.html


# Plot

In [15]:
df = result_df
df = df.set_index(['time'])

#df['d'] = df['time'].dt.date
#df['t'] = df['time'].dt.time
#df = df.set_index(['d', 't']).drop(columns = ['time'])


In [16]:
start_time = 0
end_time = 48

fig_price = go.Figure()
fig_price.add_trace(go.Scatter(x=price_df[start_time:end_time].index, y=price_df['arb_energy_price'], mode='lines', name='Energy'))
fig_price.add_trace(go.Scatter(x=price_df[start_time:end_time].index, y=price_df['reg_up_price'], mode='lines', name='Regulation Up'))
fig_price.add_trace(go.Scatter(x=price_df[start_time:end_time].index, y=price_df['reg_down_price'], mode='lines', name='Regulation Down'))
fig_price.add_trace(go.Scatter(x=price_df[start_time:end_time].index, y=price_df['pres_capacity_price'], mode='lines', name='Primary Reserve'))
fig_price.add_trace(go.Scatter(x=price_df[start_time:end_time].index, y=price_df['cres_capacity_price'], mode='lines', name='Contingency Reserve'))

fig_price.update_layout(
    title='Prices',
    xaxis_title='Time',
    yaxis_title='$/MWh',
    # xaxis=dict(
    #     tickmode='linear',
    #     dtick=86400000.0 * 30,  # 86400000.0 milliseconds in a day * 30 days
    # ),
    legend=dict(x=1.05, y=1),  # Position the legend outside the plot
    autosize=True,
    margin=dict(l=40, r=40, t=40, b=40),
)
fig_price.show()

# Create interactive plot for charging and discharging
fig_charge_discharge = go.Figure()
fig_charge_discharge.add_trace(go.Scatter(x=result_df[start_time:end_time].index, y=result_df['net_discharge'], mode='lines', name='Net Power Discharge'))
fig_charge_discharge.add_trace(go.Scatter(x=result_df[start_time:end_time].index, y=result_df['reg_up'], mode='lines', name='Regulation Up'))
fig_charge_discharge.add_trace(go.Scatter(x=result_df[start_time:end_time].index, y=result_df['reg_down'], mode='lines', name='Regulation Down'))
fig_charge_discharge.add_trace(go.Scatter(x=result_df[start_time:end_time].index, y=result_df['pres'], mode='lines', name='Primary Reserve'))
fig_charge_discharge.add_trace(go.Scatter(x=result_df[start_time:end_time].index, y=result_df['cres'], mode='lines', name='Contingency Reserve'))

fig_charge_discharge.update_layout(
    title='Charging and Discharging Schedule',
    xaxis_title='Time',
    yaxis_title='Power (MW)',
    # xaxis=dict(
    #     tickmode='linear',
    #     dtick=86400000.0 * 30,  # 86400000.0 milliseconds in a day * 30 days
    # ),
    legend=dict(x=1.05, y=1),  # Position the legend outside the plot
    autosize=True,
    margin=dict(l=40, r=40, t=40, b=40),
)
fig_charge_discharge.show()

# Create interactive plot for charging and discharging
fig_soc = go.Figure()
fig_soc.add_trace(go.Scatter(x=result_df.index[start_time:end_time], y=result_df['soc_percent']*100, mode='lines', name='net', line=dict(color='orange')))
fig_soc.update_layout(
    title='State of Charge',
    xaxis_title='Time',
    yaxis_title='SOC (%)',
    # xaxis=dict(
    #     tickmode='linear',
    #     dtick=86400000.0 * 30,  # 86400000.0 milliseconds in a day * 30 days
    # ),
    legend=dict(x=1.05, y=1),  # Position the legend outside the plot
    autosize=True,
    margin=dict(l=40, r=40, t=40, b=40),
)
fig_soc.show()